In [13]:
pip install ragas==0.1.9 -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd

In [2]:
os.chdir('../../')
os.getcwd()

'/teamspace/studios/this_studio/Legal_Expert_Contract_Advisor_Using_Precision_RAG'

In [20]:
import yaml

def load_config():
    with open("data/evaluation/raptor.json", "r") as f:
        return yaml.safe_load(f)
qa = load_config()

In [3]:
import yaml

def load_config():
    with open("data/evaluation/robinson.json", "r") as f:
        return yaml.safe_load(f)
qa = load_config()

In [21]:
qa

[{'question': 'Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?',
  'answer': 'Except in the case of fraud the Sellers have no liability for breach of representations and warranties (See section 10.01).'},
 {'question': 'Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?',
  'answer': 'No.'},
 {'question': 'How much is the escrow amount?',
  'answer': 'The escrow amount is equal to $1,000,000.'},
 {'question': 'Is escrow amount greater than the Retention Amount?',
  'answer': 'No.'},
 {'question': 'What is the purpose of the escrow?',
  'answer': 'To serve as a recourse of the Buyer in case of post-closing adjustments of the purchase price. (See section 2.07(e)).'},
 {'question': 'May the Escrow Amount serve as a recourse for the Buyer in case of breach of r

In [22]:
question = [q['question'] for q in qa]
ground_truth = [q['answer'] for q in qa]

In [24]:
# get first 3 question
three_question = question[:3]

In [25]:
three_ground_truth = ground_truth[:3]

In [27]:
three_question

['Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?',
 'Would the Sellers be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence?',
 'How much is the escrow amount?']

In [26]:
three_ground_truth

['Except in the case of fraud the Sellers have no liability for breach of representations and warranties (See section 10.01).',
 'No.',
 'The escrow amount is equal to $1,000,000.']

In [31]:
import requests

url = "http://127.0.0.1:8000/qa/no_translation_500_50_semantic_hybrid_gpt35_DataRaptor"
answers = []
contexts = []
for q in three_question:
    data = {
        "query": q
    }
    response = requests.post(url, json=data)

    print(response.json()['response'])

    answers.append(response.json()['response'])
    contexts.append(response.json()['context'])

Based on the provided context, each Seller is responsible for a breach of representations and warranties solely as to themselves and not jointly with other Sellers. This means that each Seller is individually accountable for their own representations and warranties. If a Seller has any Liability to a broker, finder, or agent with respect to the Contemplated Transactions, they are required to satisfy this Liability in full if it is disclosed in Schedule 4.06. The Sellers are not collectively responsible for each other's breaches, and their responsibility is limited to their own actions and disclosures.
Yes, the Sellers would be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence. The context specifies that the Sellers’ Representative is not liable for actions or omissions except in cases of gross negligence, bad faith, or willful misconduct. Therefore

In [32]:
answers

["Based on the provided context, each Seller is responsible for a breach of representations and warranties solely as to themselves and not jointly with other Sellers. This means that each Seller is individually accountable for their own representations and warranties. If a Seller has any Liability to a broker, finder, or agent with respect to the Contemplated Transactions, they are required to satisfy this Liability in full if it is disclosed in Schedule 4.06. The Sellers are not collectively responsible for each other's breaches, and their responsibility is limited to their own actions and disclosures.",
 "Yes, the Sellers would be responsible if after the closing it is determined that there were inaccuracies in the representation provided by them where such inaccuracies are the result of the Sellers’ gross negligence. The context specifies that the Sellers’ Representative is not liable for actions or omissions except in cases of gross negligence, bad faith, or willful misconduct. The

In [33]:
array_contexts = [[con] for con in contexts]

In [34]:
array_contexts

[['INDIVIDUAL REPRESENTATIONS AND WARRANTIES OF THE SELLERS.\n\nEach Seller severally, and not jointly, hereby represents and warrants to the Buyer, solely as to such Seller, that:\n\nOrganization.  In the case of each Seller that is not an individual, such Seller is duly organized, validly existing and in good standing (if such concept is recognized) under the laws of the jurisdiction of its organization.\n\nNo Brokers.  Except as disclosed in Schedule\xa04.06, such Seller has no Liability that is or expected to be borne by any Acquired Company or the Buyer to any broker, finder or agent with respect to the Contemplated Transactions, and such Seller agrees to satisfy in full any Liability required to be disclosed on Schedule\xa04.06.\n\n\nREPRESENTATIONS AND WARRANTIES OF THE BUYER.\n\nThe Buyer represents and warrants to the Sellers that:\n\nThe Sellers’ Representative shall not be held liable by any of the Sellers for actions or omissions in exercising or failing to exercise all or 

In [8]:
import json
# incase kernel restart reun this to save the context and answer as a json file

# save the context and answer as a json file
with open("context.json", "w") as f:
    json.dump({"context": contexts, "answer": answers}, f)

# read the json file
with open("context.json", "r") as f:
    context = json.load(f)

contexts = context['context']
answers = context['answer']

In [11]:
len(answers), len(contexts), len(question), len(ground_truth)

(14, 14, 14, 14)

In [36]:
from datasets import Dataset
# To dict
data = {
    "question": three_question,
    "answer": answers,
    "contexts": array_contexts,
    "ground_truth": three_ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [37]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 3
})

In [44]:
from ragas import evaluate
from langchain_openai import ChatOpenAI
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    llm = ChatOpenAI(temperature=0), # use gpt-4o to increase context window, but has high cost
    # metrics=[
    #     context_precision,
    #     context_recall,
    #     faithfulness,
    #     answer_relevancy,
    # ],
)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [45]:
df = result.to_pandas()

In [46]:
df.to_csv("data/evaluation/no_translation_500_50_semantic_hybrid_gpt35_DataRaptor.csv")

In [29]:
no_translation_500_50_semantic_hybrid_gpt35 = pd.read_csv("data/evaluation/no_translation_500_50_semantic_hybrid_gpt35_DataRobinson.csv", index_col=0)
no_translation_500_50_semantic_hybrid_gpt35

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,Who are the parties to the Agreement and what ...,The parties to the Agreement are the Buyer and...,"['agreement\n\nNOW THEREFORE, in consideration...",Cloud Investments Ltd. (“Company”) and Jack Ro...,1.0,1.0,0.5,0.953996
1,What is the termination notice?,The termination notice is fourteen (14) days' ...,"['Termination: Either party, at any given time...",According to section 4: 14 days for convenienc...,1.0,1.0,1.0,0.881419
2,What are the payments to the Advisor under the...,The payments to the Advisor under the Agreemen...,"[""Without derogating from the above, if it is ...",According to section 6: 1. Fees of $9 per hour...,0.0,0.0,1.0,0.900266


In [30]:
# calculate the average score of context precision per total score

context_precision_score = no_translation_500_50_semantic_hybrid_gpt35['context_precision'].sum() / no_translation_500_50_semantic_hybrid_gpt35['context_precision'].count() * 100
context_recall_score = no_translation_500_50_semantic_hybrid_gpt35['context_recall'].sum() / no_translation_500_50_semantic_hybrid_gpt35['context_recall'].count() * 100
faithfulness_score = no_translation_500_50_semantic_hybrid_gpt35['faithfulness'].sum() / no_translation_500_50_semantic_hybrid_gpt35['faithfulness'].count() * 100
answer_relevancy_score = no_translation_500_50_semantic_hybrid_gpt35['answer_relevancy'].sum() / no_translation_500_50_semantic_hybrid_gpt35['answer_relevancy'].count() * 100

avg_total_score = (context_precision_score + context_recall_score + faithfulness_score + answer_relevancy_score) / 4

avg_context_score = (context_precision_score + context_recall_score) / 2

avg_generate_score = (faithfulness_score + answer_relevancy_score) / 2

print("Average of All Scores is :: ", avg_total_score, " %")
print("Average of Context Scores is :: ", avg_context_score, " %")
print("The Avet]rage Generation Score :: ", avg_generate_score, " %")
print("Context Precision Score :: ", context_precision_score, " %")
print("Context Recall Score :: ", context_recall_score, " %")
print("Answer Relevancy Score :: ", answer_relevancy_score, " %")
print("Faithfulness Score :: ", faithfulness_score, " %")

Average of All Scores is ::  76.96401545344405  %
Average of Context Scores is ::  66.66666666333333  %
The Avet]rage Generation Score ::  87.26136424355477  %
Context Precision Score ::  66.66666665999999  %
Context Recall Score ::  66.66666666666666  %
Answer Relevancy Score ::  91.1893951537762  %
Faithfulness Score ::  83.33333333333334  %


In [47]:
no_translation_500_50_semantic_hybrid_gpt35_DataRaptor = pd.read_csv("data/evaluation/no_translation_500_50_semantic_hybrid_gpt35_DataRaptor.csv", index_col=0)
no_translation_500_50_semantic_hybrid_gpt35_DataRaptor

,question,answer,contexts,ground_truth,answer_relevancy,context_precision,faithfulness,context_recall
0,Under what circumstances and to what extent th...,"Based on the provided context, each Seller is ...",['INDIVIDUAL REPRESENTATIONS AND WARRANTIES OF...,Except in the case of fraud the Sellers have n...,0.911028,1.0,1.00,0.0
1,Would the Sellers be responsible if after the ...,"Yes, the Sellers would be responsible if after...",['Losses arising out of or caused by the Selle...,No.,0.944305,1.0,0.25,1.0
2,How much is the escrow amount?,The context provided does not specify the exac...,['Purchase Price Adjustment. If any of the Cl...,"The escrow amount is equal to $1,000,000.",0.000000,1.0,1.00,0.0


In [43]:
# calculate the average score of context precision per total score

context_precision_score = no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['context_precision'].sum() / no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['context_precision'].count() * 100
context_recall_score = no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['context_recall'].sum() / no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['context_recall'].count() * 100
faithfulness_score = no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['faithfulness'].sum() / no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['faithfulness'].count() * 100
answer_relevancy_score = no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['answer_relevancy'].sum() / no_translation_500_50_semantic_hybrid_gpt35_DataRaptor['answer_relevancy'].count() * 100

avg_total_score = (context_precision_score + context_recall_score + faithfulness_score + answer_relevancy_score) / 4

avg_context_score = (context_precision_score + context_recall_score) / 2

avg_generate_score = (faithfulness_score + answer_relevancy_score) / 2

print("Average of All Scores is :: ", avg_total_score, " %")
print("Average of Context Scores is :: ", avg_context_score, " %")
print("The Avet]rage Generation Score :: ", avg_generate_score, " %")
print("Context Precision Score :: ", context_precision_score, " %")
print("Context Recall Score :: ", context_recall_score, " %")
print("Answer Relevancy Score :: ", answer_relevancy_score, " %")
print("Faithfulness Score :: ", faithfulness_score, " %")

Average of All Scores is ::  67.60263656822967  %
Average of Context Scores is ::  66.66666666166665  %
The Avet]rage Generation Score ::  68.53860647479269  %
Context Precision Score ::  99.99999998999999  %
Context Recall Score ::  33.33333333333333  %
Answer Relevancy Score ::  62.07721294958538  %
Faithfulness Score ::  75.0  %
